# This Looks Like That There

Main training notebook.

In [1]:
import os
import sys
import time
import imp #imp.reload(module)

import numpy as np
from tqdm import trange
from icecream import ic

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import tensorflow as tf

import network
import experiment_settings 
import data_functions
import push_prototypes
import plots
import common_functions

In [2]:
__author__ = "Elizabeth A. Barnes and Randal J Barnes"
__version__ = "23 November 2021"

In [3]:
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.dpi']= 150
dpiFig = 300.

## Print the detailed system info

In [4]:
print(f"python version = {sys.version}")
print(f"numpy version = {np.__version__}")
print(f"tensorflow version = {tf.__version__}")

python version = 3.9.4 (default, Apr  9 2021, 09:32:38) 
[Clang 10.0.0 ]
numpy version = 1.20.1
tensorflow version = 2.5.0


## Define experiment settings and directories

In [5]:
EXP_NAME = 'mjo'

imp.reload(experiment_settings)
settings = experiment_settings.get_settings(EXP_NAME)

imp.reload(common_functions)
model_dir, model_diagnostics_dir, vizualization_dir = common_functions.get_exp_directories(EXP_NAME)

## Define the network parameters

In [6]:
RANDOM_SEED          = settings['random_seed']
BATCH_SIZE_PREDICT   = settings['batch_size_predict']
BATCH_SIZE           = settings['batch_size']
NLAYERS              = settings['nlayers']
NFILTERS             = settings['nfilters']   
assert(len(NFILTERS)==NLAYERS)

NCLASSES             = settings['nclasses']
PROTOTYPES_PER_CLASS = settings['prototypes_per_class']
NPROTOTYPES          = np.sum(PROTOTYPES_PER_CLASS)

NEPOCHS              = settings['nepochs']
LR_INIT              = settings['lr']
LR_CALLBACK_EPOCH    = settings['lr_cb_epoch']
PATIENCE             = 100

## Initialize

In [7]:
np.random.seed(RANDOM_SEED)
rng = np.random.default_rng(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

## Get and process the data

In [8]:
imp.reload(data_functions)
DATA_NAME = settings['data_name']
DATA_DIR = settings['data_dir']

if(EXP_NAME[:3]=='mjo'):

    labels, data, lat, lon, time = data_functions.load_mjo_data(DATA_DIR)
    X_train, y_train, time_train, X_val, y_val, time_val, X_test, y_test, time_test = data_functions.get_and_process_mjo_data(labels,
                                                                                         data,
                                                                                         time,
                                                                                         rng, 
                                                                                         colored=settings['colored'],
                                                                                         standardize=settings['standardize'],
                                                                                         shuffle=settings['shuffle'],
                                                                                        )        
elif(EXP_NAME[:9]=='quadrants'):
    filename = DATA_DIR + DATA_NAME + '.mat'
    X_train, y_train, X_val, y_val, X_test, y_test, lat, lon = data_functions.get_and_process_data(filename, 
                                                                                        rng, 
                                                                                        colored=settings['colored'],
                                                                                        standardize=settings['standardize'],
                                                                                        shuffle=settings['shuffle'],
                                                                                        )      
    

[0 1 2 3 4 5 6 7 8]


ic| years_train: array([1901, 1900, 1979, 1908, 2004, 1920, 1932, 1951, 1985, 1924, 1956,
                        1952, 2000, 1996, 1938, 1991, 1973, 1921, 1960, 1977, 1982, 1969,
                        1994, 1988, 1998, 1978, 1970, 2003, 2002, 1963, 1997, 1931, 1910,
                        1993, 1904, 1942, 1981, 1967, 1966, 1983, 1989, 2006, 1974, 1984,
                        1980, 1943, 1934, 1971, 1929, 1930, 1919, 1986, 1914, 1959, 1926,
                        1999, 1905, 1975, 1948, 1954, 1925, 1940, 1922, 1936, 1968, 1947,
                        1990, 1909, 1939, 1972, 2009, 1944, 1957, 1958, 1928, 1955, 1933,
                        1950, 1913, 2005, 1915, 1911, 1987, 1964, 1995, 2001, 1927, 1962,
                        1906])
ic| years_val: array([1902, 1903, 1907, 1912, 1916, 1917, 1918, 1923, 1935, 1937, 1941,
                      1945, 1946, 1949, 1953, 1961, 1965, 1976, 1992, 2007, 2008, 2010])
ic| years_test: 2010


raw_data        = (40422, 17, 105, 3)
training data   = (32387, 17, 105, 3), (32387,)
validation data = (8035, 17, 105, 3), (8035,)
test data       = (365, 17, 105, 3), (365,)
X_mean          = 0.00029500475989341156
X_std           = 1.0281252570023809


In [9]:
proto_class_mask = network.createClassIdentity(PROTOTYPES_PER_CLASS)

prototypes_of_correct_class_train = np.zeros((len(y_train),NPROTOTYPES))
for i in range(0,prototypes_of_correct_class_train.shape[0]):
    prototypes_of_correct_class_train[i,:] = proto_class_mask[:,int(y_train[i])]
    
prototypes_of_correct_class_val   = np.zeros((len(y_val),NPROTOTYPES))    
for i in range(0,prototypes_of_correct_class_val.shape[0]):
    prototypes_of_correct_class_val[i,:] = proto_class_mask[:,int(y_val[i])]

prototypes_of_correct_class_test   = np.zeros((len(y_test),NPROTOTYPES))    
for i in range(0,prototypes_of_correct_class_test.shape[0]):
    prototypes_of_correct_class_test[i,:] = proto_class_mask[:,int(y_test[i])]
    

## Define the training callbacks and metrics

In [10]:
# callbacks
def scheduler(epoch, lr):
    if epoch < LR_CALLBACK_EPOCH:
        return np.round(lr,8)
    else:
        if(epoch % 2 == 0):
            return lr/2.
        else:
            return lr

lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=1)    
    
es_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_sparse_categorical_accuracy', 
    mode='max',
    patience=settings['patience'], 
    restore_best_weights=True, 
    verbose=1
)

callbacks_list = [
#     lr_callback,
#     es_callback,
]            

# metrics
metrics_list = [
    tf.keras.metrics.SparseCategoricalAccuracy(),
]

## Instantiate the model

In [11]:
__ = imp.reload(network)
tf.keras.backend.clear_session()

model = network.build_model(
    nlayers              = NLAYERS,
    nfilters             = NFILTERS,
    input_shape          = X_train.shape[1:],
    output_shape         = NCLASSES,
    prototypes_per_class = PROTOTYPES_PER_CLASS,
    network_seed         = RANDOM_SEED,    
    prototype_channels   = settings['prototype_channels'],    
    coeff_cluster        = settings['coeff_cluster'],
    coeff_separation     = settings['coeff_separation'],
    coeff_l1             = settings['coeff_l1'],
    incorrect_strength   = settings['incorrect_strength'],
    double_conv          = settings['double_conv'],
    kernel_l1_coeff      = 0.0,#settings['kernel_l1_coeff'],
    kernel_l2_coeff      = 0.0,#settings['kernel_l2_coeff'],
    drop_rate            = 0.0,
    drop_rate_final      = 0.0,        
    
)
model.summary()

3
[16, 16, 16]
(17, 105, 3)
9
[10, 10, 10, 10, 10, 10, 10, 10, 10]
30
False
0.2
-0.02
0.1
-0.5
False
8
64
0.0
0.0
0.0
0.0
False
Model: "full_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 17, 105, 3)] 0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 17, 105, 16)  448         inputs[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)               (None, 17, 105, 16)  0           conv_0[0][0]                     
__________________________________________________________________________________________________
maxpooling_0 (AveragePooling2D) (None, 8, 52, 16)    0      

## Load pre-trained weights into convolutional layers

In [12]:
if(settings['pretrain'] == True):

    if(settings['pretrain_exp'] is None):
        PRETRAINED_MODEL = model_dir + 'pretrained_model_' + EXP_NAME 
    else:
        PRETRAINED_MODEL = './saved_models/' + settings['pretrain_exp'] 

    print('loading pretrained convolutional layers from ' + PRETRAINED_MODEL)
    pretrained_model = tf.keras.models.load_model(PRETRAINED_MODEL)

    for layer in range(1,len(model.layers)):
        if(model.layers[layer].name[:4]=='conv'):
            print('   loading pretrained weights for --> ' + model.layers[layer].name)
            model.layers[layer].set_weights(pretrained_model.layers[layer].get_weights())
else:
    print('no pretrained model specified. keeping random initialized weights.')
    

loading pretrained convolutional layers from ./saved_models/mjo/pretrained_model_mjo
   loading pretrained weights for --> conv_0
   loading pretrained weights for --> conv_1
   loading pretrained weights for --> conv_2


In [13]:
# raise ValueError('here')

***

# Run Training Stages

In [14]:
imp.reload(network)
imp.reload(plots)
imp.reload(push_prototypes)
imp.reload(experiment_settings)
settings = experiment_settings.get_settings(EXP_NAME)

In [15]:
ic(np.shape(X_train))
ic(np.shape(prototypes_of_correct_class_train))
ic(np.shape(prototypes_of_correct_class_train))

ic| np.shape(X_train): (32387, 17, 105, 3)
ic| np.shape(prototypes_of_correct_class_train): (32387, 90)
ic| np.shape(prototypes_of_correct_class_train): (32387, 90)


(32387, 90)

In [16]:
imp.reload(push_prototypes)
NEPOCHS    = settings['nepochs']
STAGE_LIST = (0,1,2,3,4,5,6,7,8,9)#range(len(NEPOCHS))#(1,2,3,4,5)#range(len(NEPOCHS))

for stage in STAGE_LIST:
    
    print('--------------------')
    print('TRAINING STAGE = ' + str(stage))
    print('--------------------')

    # load previously trained stage, unless it is the 0th stage
    if(stage != 0):
        tf.keras.backend.clear_session()
        model_filename = model_dir + 'model_' + EXP_NAME + '_stage' + str(stage-1)
#         model = common_functions.load_model(model_filename)
        model.load_weights(model_filename)
        
    # learn layers (during even numbered stages)
    if(stage % 2 == 0):
        # train prototypes layers (and possibly CNN layers)
        if(settings['pretrain']==False and settings['train_cnn_in_stage'] == True):
            model = network.set_trainable_layers(model, [True,True,True,False])            
        elif(settings['train_cnn_in_stage'] == False or stage==0):
            model = network.set_trainable_layers(model, [False,True,True,False])
        elif(settings['train_cnn_in_stage'] == True):
            model = network.set_trainable_layers(model, [True,True,True,False])            
        elif(stage >= settings['train_cnn_in_stage']):
            model = network.set_trainable_layers(model, [True,True,True,False])            
        else:
            model = network.set_trainable_layers(model, [False,True,True,False])
    else:
        #.......................................................
        # push the prototypes
        #.......................................................        
        model, push_info = push_prototypes.push(model, 
                                                [X_train,prototypes_of_correct_class_train], 
                                                prototypes_of_correct_class_train, 
                                                perform_push=True,
                                                batch_size=BATCH_SIZE_PREDICT,
                                                verbose=False,
                                               )        
        print('Push complete.\n')            

        # train weights layer only
        model = network.set_trainable_layers(model, [False,False,False,True])        

    #.......................................................
    # compile the model
    #.......................................................
    if(stage>=settings['cut_lr_stage']):
        lr_factor = 10.**(np.floor((stage-settings['cut_lr_stage']+2)/2))
    else:
        lr_factor = 1.
    if(LR_INIT/lr_factor<settings['min_lr']):
        lr_factor = LR_INIT/settings['min_lr']
    print('learning rate = ' + str(np.asarray(LR_INIT/lr_factor,dtype='float32')))

    # compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=np.asarray(LR_INIT/lr_factor,dtype='float32'), 
        ),
        loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics = metrics_list,
    )
#     model.summary()
    ic(np.min(model.layers[-3].get_weights()[1]),np.max(model.layers[-3].get_weights()[1]))

    #.......................................................
    # train the model
    #.......................................................
    print('Training the model...')    
    
    tf.random.set_seed(RANDOM_SEED)   
    np.random.seed(RANDOM_SEED)    
    history = model.fit(
        [X_train,prototypes_of_correct_class_train],
        y_train,
        validation_data=([[X_val,prototypes_of_correct_class_val]], [y_val]),
        batch_size=BATCH_SIZE,
        epochs=NEPOCHS[stage],
        shuffle=True,
        verbose=1,
        callbacks=callbacks_list
    )
    print('Training complete.\n')            
        

    # save the model at this training stage
    model_filename = model_dir + 'model_' + EXP_NAME + '_stage' + str(stage)
    common_functions.save_model(model, model_filename) 
    
    #.......................................................
    # plot results
    #.......................................................  
    try:
        # plot loss history of the model
        plots.plot_loss_history(history)
        plt.savefig(model_diagnostics_dir + EXP_NAME + '_loss_history_stage' + str(stage) + '.png', dpi=dpiFig)    
        plt.close()

        # plot the weights
        plots.plot_weights(model, PROTOTYPES_PER_CLASS)    
        plt.savefig(model_diagnostics_dir + EXP_NAME + '_weights_stage' + str(stage) + '.png', dpi=dpiFig)
        plt.close()
    except:
        print('not making plots...')
        plt.close()

--------------------
TRAINING STAGE = 0
--------------------
   conv_0 --> False
   maxpooling_0 --> False
   conv_1 --> False
   maxpooling_1 --> False
   conv_2 --> False
   maxpooling_2 --> False
   first_1x1_conv --> True
   second_1x1_conv --> True
   prototype --> True
   final_weights --> False
learning rate = 0.01


ic| np.min(model.layers[-3].get_weights()[1]): 0.0
    np.max(model.layers[-3].get_weights()[1]): 0.0


Training the model...
Epoch 1/10
1013/1013 [==============================] - 13s 12ms/step - loss: 37.5770 - sparse_categorical_accuracy: 0.4273 - cluster_cost: 0.6228 - separation_cost: 0.4787 - l1_weights_cost: 36.0000 - val_loss: 37.3056 - val_sparse_categorical_accuracy: 0.5039 - val_cluster_cost: 0.3394 - val_separation_cost: 0.2674 - val_l1_weights_cost: 36.0000
Epoch 2/10
1013/1013 [==============================] - 10s 10ms/step - loss: 37.1337 - sparse_categorical_accuracy: 0.5606 - cluster_cost: 0.3210 - separation_cost: 0.2638 - l1_weights_cost: 36.0000 - val_loss: 37.1140 - val_sparse_categorical_accuracy: 0.5735 - val_cluster_cost: 0.3273 - val_separation_cost: 0.2649 - val_l1_weights_cost: 36.0000
Epoch 3/10
1013/1013 [==============================] - 11s 11ms/step - loss: 37.0668 - sparse_categorical_accuracy: 0.5887 - cluster_cost: 0.3227 - separation_cost: 0.2710 - l1_weights_cost: 36.0000 - val_loss: 37.1007 - val_sparse_categorical_accuracy: 0.5775 - val_cluster_co

ic| np.min(model.layers[-3].get_weights()[1]): -3.3630624
    np.max(model.layers[-3].get_weights()[1]): 2.9702795


Performing push of prototypes.
Push complete.

   conv_0 --> False
   maxpooling_0 --> False
   conv_1 --> False
   maxpooling_1 --> False
   conv_2 --> False
   maxpooling_2 --> False
   first_1x1_conv --> False
   second_1x1_conv --> False
   prototype --> False
   final_weights --> True
learning rate = 0.01
Training the model...
Epoch 1/10
1013/1013 [==============================] - 8s 7ms/step - loss: 5.4719 - sparse_categorical_accuracy: 0.5235 - cluster_cost: 0.0673 - separation_cost: 0.0290 - l1_weights_cost: 2.7896 - val_loss: 2.2450 - val_sparse_categorical_accuracy: 0.4597 - val_cluster_cost: 0.0688 - val_separation_cost: 0.0304 - val_l1_weights_cost: 0.4915
Epoch 2/10
1013/1013 [==============================] - 6s 6ms/step - loss: 1.8040 - sparse_categorical_accuracy: 0.5042 - cluster_cost: 0.0674 - separation_cost: 0.0290 - l1_weights_cost: 0.4215 - val_loss: 1.9138 - val_sparse_categorical_accuracy: 0.4630 - val_cluster_cost: 0.0688 - val_separation_cost: 0.0304 - val_l1

...

Performing push of prototypes.
Push complete.

   conv_0 --> False
   maxpooling_0 --> False
   conv_1 --> False
   maxpooling_1 --> False
   conv_2 --> False
   maxpooling_2 --> False
   first_1x1_conv --> False
   second_1x1_conv --> False
   prototype --> False
   final_weights --> True
learning rate = 1e-04
Training the model...
Epoch 1/10
1013/1013 [==============================] - 7s 7ms/step - loss: 0.8782 - sparse_categorical_accuracy: 0.7513 - cluster_cost: 0.0039 - separation_cost: 7.5693e-04 - l1_weights_cost: 0.1403 - val_loss: 0.9811 - val_sparse_categorical_accuracy: 0.7246 - val_cluster_cost: 0.0040 - val_separation_cost: 8.0962e-04 - val_l1_weights_cost: 0.1310
Epoch 2/10
1013/1013 [==============================] - 6s 6ms/step - loss: 0.8285 - sparse_categorical_accuracy: 0.7504 - cluster_cost: 0.0039 - separation_cost: 7.5707e-04 - l1_weights_cost: 0.1253 - val_loss: 0.9601 - val_sparse_categorical_accuracy: 0.7197 - val_cluster_cost: 0.0040 - val_separation_cost: 8.

ic| np.min(model.layers[-3].get_weights()[1]): -5.5549974
    np.max(model.layers[-3].get_weights()[1]): 5.7040596


--------------------
TRAINING STAGE = 8
--------------------
   conv_0 --> True
   maxpooling_0 --> True
   conv_1 --> True
   maxpooling_1 --> True
   conv_2 --> True
   maxpooling_2 --> True
   first_1x1_conv --> True
   second_1x1_conv --> True
   prototype --> True
   final_weights --> False
learning rate = 1e-05
Training the model...
Epoch 1/10
1013/1013 [==============================] - 26s 24ms/step - loss: 0.7115 - sparse_categorical_accuracy: 0.7527 - cluster_cost: 0.0043 - separation_cost: 8.7273e-04 - l1_weights_cost: 0.0968 - val_loss: 0.7945 - val_sparse_categorical_accuracy: 0.7349 - val_cluster_cost: 0.0045 - val_separation_cost: 0.0010 - val_l1_weights_cost: 0.0968
Epoch 2/10
1013/1013 [==============================] - 26s 26ms/step - loss: 0.6744 - sparse_categorical_accuracy: 0.7715 - cluster_cost: 0.0047 - separation_cost: 0.0010 - l1_weights_cost: 0.0968 - val_loss: 0.7693 - val_sparse_categorical_accuracy: 0.7400 - val_cluster_cost: 0.0048 - val_separation_cost: 

ic| np.min(model.layers[-3].get_weights()[1]): -5.5549383
    np.max(model.layers[-3].get_weights()[1]): 5.706313


Performing push of prototypes.
Push complete.

   conv_0 --> False
   maxpooling_0 --> False
   conv_1 --> False
   maxpooling_1 --> False
   conv_2 --> False
   maxpooling_2 --> False
   first_1x1_conv --> False
   second_1x1_conv --> False
   prototype --> False
   final_weights --> True
learning rate = 1e-05
Training the model...
Epoch 1/10
1013/1013 [==============================] - 7s 7ms/step - loss: 0.7429 - sparse_categorical_accuracy: 0.7542 - cluster_cost: 0.0032 - separation_cost: 6.3923e-04 - l1_weights_cost: 0.0948 - val_loss: 0.8670 - val_sparse_categorical_accuracy: 0.7271 - val_cluster_cost: 0.0033 - val_separation_cost: 6.7898e-04 - val_l1_weights_cost: 0.0941
Epoch 2/10
1013/1013 [==============================] - 6s 6ms/step - loss: 0.7372 - sparse_categorical_accuracy: 0.7579 - cluster_cost: 0.0032 - separation_cost: 6.3952e-04 - l1_weights_cost: 0.0933 - val_loss: 0.8622 - val_sparse_categorical_accuracy: 0.7289 - val_cluster_cost: 0.0033 - val_separation_cost: 6.